In [339]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

session = SnorkelSession()
dev_cands = session.query(Spouse).filter(Spouse.split == 1).all()
test_cands = session.query(Spouse).filter(Spouse.split == 2).all()

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]
gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

In [302]:
import numpy as np
unique, counts = np.unique(gold_labels_dev, return_counts=True)
print(dict(zip(unique, counts)))


unique, counts = np.unique(gold_labels_test, return_counts=True)
print(dict(zip(unique, counts)))

print(len(gold_labels_dev),len(gold_labels_test))

{0: 2625, 1: 189}
{0: 2484, 1: 218}
2814 2702


In [262]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [259]:
# ####### Discrete ##########

# import re
# from snorkel.lf_helpers import (
#     get_left_tokens, get_right_tokens, get_between_tokens,
#     get_text_between, get_tagged_text,
# )

# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
# family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# # Helper function to get last name
# def last_name(s):
#     name_parts = s.split(' ')
#     return name_parts[-1] if len(name_parts) > 1 else None    

# def LF_husband_wife(c):
#     return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_husband_wife_left_window(c):
#     if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (1,1)
#     elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (1,1)
#     else:
#         return (0,0)
    
# def LF_same_last_name(c):
#     p1_last_name = last_name(c.person1.get_span())
#     p2_last_name = last_name(c.person2.get_span())
#     if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
#         if c.person1.get_span() != c.person2.get_span():
#             return (1,1)
#     return (0,0)

# def LF_no_spouse_in_sentence(c):
#     return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

# def LF_and_married(c):
#     return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
# def LF_familial_relationship(c):
#     return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_family_left_window(c):
#     if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (-1,1)
#     elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_other_relationship(c):
#     return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


# import bz2

# # Function to remove special characters from text
# def strip_special(s):
#     return ''.join(c for c in s if ord(c) < 128)

# # Read in known spouse pairs and save as set of tuples
# with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
#     known_spouses = set(
#         tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
#     )
# # Last name pairs for known spouses
# last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
# def LF_distant_supervision(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

# def LF_distant_supervision_last_names(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     p1n, p2n = last_name(p1), last_name(p2)
#     return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


# LFs = [
#     LF_distant_supervision, LF_distant_supervision_last_names, 
#     LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
#     LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
#     LF_family_left_window, LF_other_relationship
# ]

In [261]:
##### Continuous ################


import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)


spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

# def LF_other_relationship_left_window(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
#     return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    s = s.decode("utf-8") 
    return ''.join(c for c in s if (ord(c) < 128))


# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'r') as f:
    known_spouses = set(
        tuple(strip_special(x).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

# def LF_Three_Lists_Left_Window(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife_left_window(c)
#     c2,s2 = LF_family_left_window(c)
#     c3,s3 = LF_other_relationship_left_window(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'Left:',s1,s2,s3,prob_sc
    
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1

# def LF_Three_Lists_Between_Words(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife(c)
#     c2,s2 = LF_familial_relationship(c)
#     c3,s3 = LF_other_relationship(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
    
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'BW:',s1,s2,s3,prob_sc
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1
    
LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [257]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]
    
output shape : [NoOfDataPoints,2,NoOfLFs]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [263]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

# dev_L_S = get_L_S_Tensor(dev_cands)

# np.save("dev_L_S_smooth",np.array(dev_L_S))

# train_L_S = get_L_S_Tensor(train_cands)
# np.save("train_L_S_smooth",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_smooth",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 16-6-2018, 12:11:51
500data points labelled in 0.5303041696548462 mins
1000data points labelled in 1.0592769702275595 mins
1500data points labelled in 1.6191872398058573 mins
2000data points labelled in 2.2701824069023133 mins
2500data points labelled in 2.8335480292638144 mins
--- 187.22964453697205 seconds ---


In [144]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
    print(precision_recall_fscore_support(gold_labels_test,pl,average='binary'))
    print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
#     return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [340]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]
import numpy as np
dev_L_S = np.load("dev_L_S_smooth.npy")
test_L_S = np.load("test_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")
# true_labels = gold_labels_test
print(len(gold_labels_dev),len(gold_labels_test))
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

2814 2702
(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [279]:
dev_L_S = np.load("dev_L_S_discrete.npy")
test_L_S = np.load("test_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

In [265]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [161]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]

def merge(a,b):
    c = []
    for i in range(len(a)):
        ci = []
        ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
        ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
        ci.append(ci_l)
        ci.append(ci_s)
        c.append(ci)
    return c
import numpy as np
dev_L_S_s = np.load("dev_L_S_smooth.npy")
train_L_S_s = np.load("train_L_S_smooth.npy")

dev_L_S_d = np.load("dev_L_S_discrete.npy")
train_L_S_d = np.load("train_L_S_discrete.npy")

dev_L_S = np.array(merge(dev_L_S_d,dev_L_S_s))
train_L_S = np.array(merge(train_L_S_d,train_L_S_s))

LF_l = LF_l + LF_l
print(len(LF_l))


print(dev_L_S.shape,train_L_S.shape)

20
(2814, 2, 20) (22276, 2, 20)


In [266]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

10


In [350]:

def train(lr,ep,th,af,batch_size=32,LF_acc=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)
        
        

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(debug):
            print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        if(debug):
            print("s",s)
            print("l",l)
#         print(s.graph)
        if(smooth):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
        if(debug):
            print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        if(debug):
            print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        if(debug):
            print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            if(debug):
                print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            if(debug):
                print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")
            
        ### for precision penalty
        def softplus(j):
            Lj = tf.map_fn(lambda li : tf.gather(li,j),l)
            if(debug):
                print("sft Lj",Lj)
            kj = tf.gather(k,j)
            if(debug):
                print("sft kj",kj)
            aj = tf.gather(LF_a,j)
            if(debug):
                print("sft aj",aj)
            indices = tf.where(tf.equal(Lj,kj))
            if(debug):
                print("sft indices",indices)
            li_lij_eq_kj = tf.gather(l,tf.squeeze(indices,1))
            if(smooth):
                si_lij_eq_kj = tf.gather(s_,tf.squeeze(indices,1))
            if(debug):
                print("sft l_ij_eq_kj",li_lij_eq_kj)
            if(smooth):
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                prec_t_pout = (tf.matmul(li_lij_eq_kj*si_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            else:
                prec_t_pout = (tf.matmul(li_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            f =  tf.reduce_sum(aj - prec_t_pout)
            if(debug):
                print("f",f)
            sft = tf.nn.softplus(f,name="sft")
            if(debug):
                print("sft",sft)
            return sft
        
#         logsft = tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
#         sft  =  tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
        
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        if(debug):
            print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        if(debug):
            print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        if(debug):
            print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk))))
        elif(penalty == 4):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
  
        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
       
        if(debug):
            print("loss",loss)
        tf.summary.scalar('loss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)
                print("dev set")
                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary"))
                print()
                
#             # Initialize an iterator over the validation dataset.
#             sess.run(dev_init_op)
#             a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
#             print(t)

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))

#             print("acc",accuracy_score(true_labels,pl))

# #             predictAndPrint(pl)
#             print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [252]:
a = np.empty([len(LF_l)],dtype=np.float64)
a.fill(0.5)
a

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [331]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import numpy as np
def get_LF_acc(L_S,true_labels):
    #L_S : a numpy array of [NoOfDataPoints,2,NoOfLFs] 
    #true_labels : numpy array [NoOfDataPoints]
    
    tl = [-1 if x==0 else x for x in true_labels]
    unique, counts = np.unique(tl, return_counts=True)
    print(dict(zip(unique, counts)))

    # take only labels 
    L_S = L_S[:,0,:]
    #L_S shape [NoOfDataPoints,NoOfLFs]
    LF_acc = []
    for i in range(L_S.shape[1]):
#         print(accuracy_score(L_S[:,i],tl,normalize=False),accuracy_score(L_S[:,i],tl))
        LF_acc.append(accuracy_score(L_S[:,i],tl))
#         unique, counts = np.unique(L_S[:,i], return_counts=True)
#         print(i,dict(zip(unique, counts)))
#         print(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
#         LF_acc.append(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
    return np.array(LF_acc)
                      
get_LF_acc(dev_L_S,gold_labels_dev)

{1: 189, -1: 2625}


array([7.10732054e-04, 2.48756219e-03, 3.30490405e-02, 2.87846482e-02,
       6.75195451e-03, 5.99502488e-01, 3.55366027e-04, 1.12651031e-01,
       7.46268657e-02, 7.46268657e-03])

In [357]:
acc = np.array([7.10732054e-04, 2.48756219e-03, 3.30490405e-02, 2.87846482e-02,\
       6.75195451e-03, 5.99502488e-01, 3.55366027e-04, 1.12651031e-01,\
       7.46268657e-02, 7.46268657e-03])
### smooth LFs with acc on discrete LFs
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=4,debug=False)

batch-size: 512 alpha-init: 0.0
precision penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 170217.75416025025
dev set
[ 0.00137266 -0.00166069  0.00029264  0.00020329  0.00126115  0.00021667
  0.0018891   0.00011512  0.00126639  0.00045027]
[[1.11734745 0.81401745 1.00936248 1.0004271  1.10618046 1.04153514
  1.16900881 1.03140499 1.14653194 1.06492007]]
{0: 2732, 1: 82}
acc 0.9243070362473348
(0.35365853658536583, 0.15343915343915343, 0.21402214022140223, None)

test set
{0: 2631, 1: 71}
acc 0.9115470022205774
(0.352112676056338, 0.11467889908256881, 0.17301038062283738, None)

1 loss 170166.99521960263
dev set
[ 1.56889740e-03 -1.46446395e-03  4.89001812e-04  3.99647685e-04
  1.45749166e-03  2.04943991e-05  2.08545759e-03 -8.12470987e-05
  1.07003415e-03  2.53911117e-04]
[[1.11715121 0.8138212  1.00916604 1.00023069 1.10598405 1.04173127
  1.16881242 1.03160143 1.14672836 1.0651164 ]]
{0: 2733, 1: 81}
acc 0.9246624022743426
(0.35802469135802467, 0.15343915343915343, 0.2148148148148148, None)

test set
{0: 2631, 1: 71}
acc 0.9115470022205774
(0.352112676056338, 0.114

2 loss 167752.1774122239
dev set
[0.30176488 0.29873142 0.30068558 0.30059623 0.30165372 0.29982425
 0.30228172 0.29972212 0.30087341 0.30005728]
[[1.11695526 0.81362542 1.00896929 1.00003398 1.10578779 1.04192769
  1.16861598 1.03179836 1.14692524 1.06531275]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2626, 1: 76}
acc 0.9111769059955589
(0.35526315789473684, 0.12385321100917432, 0.1836734693877551, None)

3 loss 167723.98865562686
dev set
[0.30196087 0.29892732 0.30088246 0.3007931  0.30184981 0.29962762
 0.30247834 0.29952509 0.30067638 0.29986025]
[[1.11675923 0.81342945 1.00877244 0.99983718 1.10559161 1.04212441
  1.1684195  1.03199546 1.14712228 1.06550981]]
{0: 2728, 1: 86}
acc 0.9250177683013504
(0.37209302325581395, 0.1693121693121693, 0.23272727272727273, None)

test set
{0: 2628, 1: 74}
acc 0.9104367135455218
(0.33783783783783783, 0.11467889908256881, 0.17123287671232879, None)

4 loss 167695.76306

4 loss 168045.79094114547
dev set
[0.60215571 0.59912124 0.60107983 0.60098922 0.60204481 0.59942947
 0.60267385 0.5993272  0.60047888 0.599663  ]
[[1.11656489 0.81323737 1.00857616 0.99964281 1.1053981  1.04232307
  1.1682247  1.0321894  1.14730748 1.06569887]]
{0: 2680, 1: 134}
acc 0.910092395167022
(0.26119402985074625, 0.18518518518518517, 0.21671826625386997, None)

test set
{0: 2572, 1: 130}
acc 0.8985936343449297
(0.2846153846153846, 0.16972477064220184, 0.21264367816091956, None)

batch-size: 512 alpha-init: 0.7000000000000001
precision penalty
0 loss 168266.52989743015
dev set
[0.7013721  0.69833762 0.70029272 0.70020336 0.70126016 0.70021629
 0.70188902 0.70011522 0.70126896 0.7004537 ]
[[1.11734838 0.81402122 1.00936226 1.00042787 1.1061834  1.04153564
  1.16900887 1.03140258 1.14651948 1.06484031]]
{0: 2551, 1: 263}
acc 0.9076048329779673
(0.3650190114068441, 0.5079365079365079, 0.4247787610619469, None)

test set
{0: 2375, 1: 327}
acc 0.8937823834196891
(0.3944954128440367

...

4 loss 167699.79320817365
dev set
[0.50166724 0.49863385 0.50058796 0.5004985  0.50155596 0.49992127
 0.50218407 0.49981175 0.5009711  0.50015488]
[[1.11705292 0.81372301 1.00906689 1.00013172 1.10588566 1.04183075
  1.16871399 1.03170085 1.14682744 1.06521511]]
{0: 2703, 1: 111}
acc 0.9189765458422174
(0.32432432432432434, 0.19047619047619047, 0.24, None)

test set
{0: 2604, 1: 98}
acc 0.9082161361954109
(0.3469387755102041, 0.1559633027522936, 0.21518987341772153, None)

batch-size: 1024 alpha-init: 0.6000000000000001
precision penalty
0 loss 168020.99829953653
dev set
[0.60127403 0.59824059 0.60019407 0.60010466 0.60116238 0.60031491
 0.60179122 0.60021371 0.60136497 0.60054893]
[[1.11744611 0.81411629 1.0094608  1.00052541 1.10627939 1.04143702
  1.16910718 1.03130682 1.14643372 1.06482091]]
{0: 2674, 1: 140}
acc 0.9079601990049752
(0.25, 0.18518518518518517, 0.2127659574468085, None)

test set
{0: 2561, 1: 141}
acc 0.8945225758697262
(0.2624113475177305, 0.16972477064220184, 0.206

{0: 2308, 1: 394}
acc 0.8852701702442635
(0.383248730964467, 0.6926605504587156, 0.4934640522875816, None)

1 loss 168150.72579593296
dev set
[0.90136949 0.89829089 0.89992546 0.89982806 0.90115729 0.90061037
 0.90188854 0.90013334 0.90129193 0.90078802]
[[1.11735391 0.81419962 1.00975426 1.00082061 1.10645957 1.04114161
  1.16901    1.03140165 1.14652744 1.06462482]]
{0: 2489, 1: 325}
acc 0.8933901918976546
(0.3292307692307692, 0.5661375661375662, 0.41634241245136183, None)

test set
{0: 2308, 1: 394}
acc 0.8852701702442635
(0.383248730964467, 0.6926605504587156, 0.4934640522875816, None)

2 loss 168150.65754467665
dev set
[0.90146656 0.89836335 0.89984216 0.89974157 0.90120881 0.90070866
 0.90198653 0.90004793 0.90120951 0.90086667]
[[1.11725855 0.81419686 1.00985103 1.00091772 1.10650159 1.04104339
  1.16891235 1.03149719 1.14662233 1.06456665]]
{0: 2489, 1: 325}
acc 0.8933901918976546
(0.3292307692307692, 0.5661375661375662, 0.41634241245136183, None)

test set
{0: 2308, 1: 394}
ac

2 loss 168910.74129737393
dev set
[0.10132329 0.09829    0.10024342 0.10015406 0.10121168 0.10026564
 0.10183988 0.10016427 0.10131554 0.10049943]
[[1.11739679 0.81406669 1.00941179 1.00047645 1.10622995 1.04148628
  1.16905815 1.0313557  1.14648257 1.06487106]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2629, 1: 73}
acc 0.9108068097705403
(0.3424657534246575, 0.11467889908256881, 0.1718213058419244, None)

3 loss 168900.04857026113
dev set
[0.10137255 0.09833926 0.10029269 0.10020334 0.10126095 0.10021639
 0.10188914 0.100115   0.10126627 0.10045015]
[[1.11734753 0.81401742 1.0093625  1.00042717 1.10618067 1.04153553
  1.16900888 1.03140499 1.14653185 1.06492034]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2629, 1: 73}
acc 0.9108068097705403
(0.3424657534246575, 0.11467889908256881, 0.1718213058419244, None)

4 loss 168889.3528784412

4 loss 167593.00196878647
dev set
[0.40142176 0.39838848 0.40034195 0.4002526  0.40131023 0.40016715
 0.40193833 0.40006574 0.40121701 0.4004009 ]
[[1.11729834 0.81396821 1.00931317 1.00037786 1.10613139 1.04158479
  1.16895963 1.03145435 1.14658115 1.06496952]]
{0: 2717, 1: 97}
acc 0.9225302061122956
(0.35051546391752575, 0.17989417989417988, 0.23776223776223773, None)

test set
{0: 2616, 1: 86}
acc 0.9111769059955589
(0.37209302325581395, 0.14678899082568808, 0.2105263157894737, None)

batch-size: 2048 alpha-init: 0.5
precision penalty
0 loss 167701.4192528906
dev set
[0.50122477 0.49819146 0.50014488 0.50005552 0.50111312 0.50036412
 0.50174139 0.50026281 0.50141409 0.50059797]
[[1.11749531 0.81416524 1.0095103  1.00057494 1.10632855 1.04138782
  1.16915662 1.03125722 1.14638411 1.06477249]]
{0: 2701, 1: 113}
acc 0.9189765458422174
(0.3274336283185841, 0.19576719576719576, 0.24503311258278143, None)

test set
{0: 2603, 1: 99}
acc 0.9078460399703923
(0.3434343434343434, 0.15596330275

{0: 2289, 1: 413}
acc 0.8782383419689119
(0.36561743341404357, 0.6926605504587156, 0.47860538827258314, None)

1 loss 168137.47156549103
dev set
[0.80127384 0.79824018 0.80019294 0.80010301 0.80116171 0.80031476
 0.80179067 0.80022528 0.80139495 0.80074444]
[[1.11744639 0.81411768 1.00948721 1.00055072 1.10628263 1.04143717
  1.16910745 1.03130351 1.14642696 1.06463056]]
{0: 2473, 1: 341}
acc 0.892679459843639
(0.3343108504398827, 0.6031746031746031, 0.430188679245283, None)

test set
{0: 2289, 1: 413}
acc 0.8782383419689119
(0.36561743341404357, 0.6926605504587156, 0.47860538827258314, None)

2 loss 168137.27545221482
dev set
[0.801323   0.79828923 0.80024121 0.80015093 0.80121078 0.80026552
 0.80183992 0.80018278 0.80136125 0.800793  ]
[[1.1173973  0.81406917 1.00945259 1.00051653 1.10623454 1.0414864
  1.16905824 1.03135067 1.14647248 1.06458394]]
{0: 2473, 1: 341}
acc 0.892679459843639
(0.3343108504398827, 0.6031746031746031, 0.430188679245283, None)

test set
{0: 2289, 1: 413}
acc

In [359]:
a = np.empty([len(LF_l)],dtype=np.float64)
a.fill(0.5)
### smooth LFs with acc on discrete LFs
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = a ,pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=4,debug=False)

batch-size: 512 alpha-init: 0.0
precision penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 223671.90833237345
dev set
[ 0.00137266 -0.00166069  0.00029264  0.00020329  0.00126115  0.00021667
  0.0018891   0.00011512  0.00126639  0.00045027]
[[1.11734745 0.81401745 1.00936248 1.0004271  1.10618046 1.04153514
  1.16900881 1.03140499 1.14653194 1.06492007]]
{0: 2732, 1: 82}
acc 0.9243070362473348
(0.35365853658536583, 0.15343915343915343, 0.21402214022140223, None)

test set
{0: 2631, 1: 71}
acc 0.9115470022205774
(0.352112676056338, 0.11467889908256881, 0.17301038062283738, None)

1 loss 223621.1521673154
dev set
[ 1.56889742e-03 -1.46446394e-03  4.89001811e-04  3.99647684e-04
  1.45749178e-03  2.04943532e-05  2.08545761e-03 -8.12470719e-05
  1.07003418e-03  2.53911144e-04]
[[1.11715121 0.8138212  1.00916604 1.00023069 1.10598405 1.04173127
  1.16881242 1.03160143 1.14672836 1.0651164 ]]
{0: 2733, 1: 81}
acc 0.9246624022743426
(0.35802469135802467, 0.15343915343915343, 0.2148148148148148, None)

test set
{0: 2631, 1: 71}
acc 0.9115470022205774
(0.352112676056338, 0.1146

2 loss 221206.4276898442
dev set
[0.30176488 0.29873142 0.30068558 0.30059623 0.30165372 0.29982425
 0.30228172 0.29972212 0.30087341 0.30005728]
[[1.11695526 0.81362542 1.00896929 1.00003398 1.10578779 1.04192769
  1.16861598 1.03179836 1.14692524 1.06531275]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2626, 1: 76}
acc 0.9111769059955589
(0.35526315789473684, 0.12385321100917432, 0.1836734693877551, None)

3 loss 221178.24207141812
dev set
[0.30196087 0.29892732 0.30088246 0.3007931  0.30184981 0.29962762
 0.30247834 0.29952509 0.30067638 0.29986025]
[[1.11675923 0.81342945 1.00877244 0.99983718 1.10559161 1.04212441
  1.1684195  1.03199546 1.14712228 1.06550981]]
{0: 2728, 1: 86}
acc 0.9250177683013504
(0.37209302325581395, 0.1693121693121693, 0.23272727272727273, None)

test set
{0: 2628, 1: 74}
acc 0.9104367135455218
(0.33783783783783783, 0.11467889908256881, 0.17123287671232879, None)

4 loss 221150.01962

4 loss 221499.83465498377
dev set
[0.60215571 0.59912124 0.60107983 0.60098922 0.60204481 0.59942947
 0.60267385 0.5993272  0.60047888 0.599663  ]
[[1.11656489 0.81323738 1.00857616 0.99964281 1.1053981  1.04232307
  1.1682247  1.0321894  1.14730747 1.06569887]]
{0: 2680, 1: 134}
acc 0.910092395167022
(0.26119402985074625, 0.18518518518518517, 0.21671826625386997, None)

test set
{0: 2572, 1: 130}
acc 0.8985936343449297
(0.2846153846153846, 0.16972477064220184, 0.21264367816091956, None)

batch-size: 512 alpha-init: 0.7000000000000001
precision penalty
0 loss 221720.33448239003
dev set
[0.7013721  0.69833762 0.70029272 0.70020336 0.70126016 0.70021629
 0.70188902 0.70011522 0.70126896 0.7004537 ]
[[1.11734839 0.81402123 1.00936226 1.00042787 1.1061834  1.04153564
  1.16900887 1.03140257 1.14651946 1.0648401 ]]
{0: 2551, 1: 263}
acc 0.9076048329779673
(0.3650190114068441, 0.5079365079365079, 0.4247787610619469, None)

test set
{0: 2375, 1: 327}
acc 0.8937823834196891
(0.3944954128440367

...

4 loss 221192.5150835845
dev set
[0.50166724 0.49863385 0.50058796 0.5004985  0.50155596 0.49992127
 0.50218407 0.49981175 0.5009711  0.50015488]
[[1.11705292 0.81372301 1.00906689 1.00013172 1.10588566 1.04183075
  1.16871399 1.03170085 1.14682744 1.06521511]]
{0: 2703, 1: 111}
acc 0.9189765458422174
(0.32432432432432434, 0.19047619047619047, 0.24, None)

test set
{0: 2604, 1: 98}
acc 0.9082161361954109
(0.3469387755102041, 0.1559633027522936, 0.21518987341772153, None)

batch-size: 1024 alpha-init: 0.6000000000000001
precision penalty
0 loss 221513.53870640005
dev set
[0.60127403 0.59824059 0.60019407 0.60010466 0.60116238 0.60031491
 0.60179122 0.60021371 0.60136497 0.60054893]
[[1.11744611 0.81411629 1.0094608  1.00052541 1.10627939 1.04143702
  1.16910718 1.03130682 1.14643372 1.06482091]]
{0: 2674, 1: 140}
acc 0.9079601990049752
(0.25, 0.18518518518518517, 0.2127659574468085, None)

test set
{0: 2561, 1: 141}
acc 0.8945225758697262
(0.2624113475177305, 0.16972477064220184, 0.2061

{0: 2308, 1: 394}
acc 0.8852701702442635
(0.383248730964467, 0.6926605504587156, 0.4934640522875816, None)

1 loss 221642.73718323125
dev set
[0.90136948 0.89829043 0.89992529 0.89982796 0.9011561  0.90061037
 0.90188854 0.90013346 0.90129204 0.90078818]
[[1.11735393 0.81420082 1.00975427 1.00082061 1.10646047 1.04114161
  1.16901    1.03140164 1.14652744 1.06462458]]
{0: 2489, 1: 325}
acc 0.8933901918976546
(0.3292307692307692, 0.5661375661375662, 0.41634241245136183, None)

test set
{0: 2308, 1: 394}
acc 0.8852701702442635
(0.383248730964467, 0.6926605504587156, 0.4934640522875816, None)

2 loss 221642.66895963394
dev set
[0.90146655 0.89836264 0.8998419  0.89974143 0.90120704 0.90070866
 0.90198653 0.90004812 0.90120967 0.90086689]
[[1.11725858 0.81419865 1.00985103 1.00091773 1.10650298 1.04104339
  1.16891236 1.03149718 1.14662233 1.06456629]]
{0: 2489, 1: 325}
acc 0.8933901918976546
(0.3292307692307692, 0.5661375661375662, 0.41634241245136183, None)

test set
{0: 2308, 1: 394}
ac

2 loss 222426.25031324377
dev set
[0.10132329 0.09829    0.10024342 0.10015406 0.10121168 0.10026564
 0.10183988 0.10016427 0.10131554 0.10049943]
[[1.11739679 0.81406669 1.00941179 1.00047645 1.10622995 1.04148628
  1.16905815 1.0313557  1.14648257 1.06487106]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2629, 1: 73}
acc 0.9108068097705403
(0.3424657534246575, 0.11467889908256881, 0.1718213058419244, None)

3 loss 222415.55829435174
dev set
[0.10137255 0.09833926 0.10029269 0.10020334 0.10126095 0.10021639
 0.10188914 0.100115   0.10126627 0.10045016]
[[1.11734753 0.81401742 1.0093625  1.00042717 1.10618067 1.04153553
  1.16900888 1.03140499 1.14653185 1.06492034]]
{0: 2727, 1: 87}
acc 0.9246624022743426
(0.367816091954023, 0.1693121693121693, 0.2318840579710145, None)

test set
{0: 2629, 1: 73}
acc 0.9108068097705403
(0.3424657534246575, 0.11467889908256881, 0.1718213058419244, None)

4 loss 222404.8633109179

4 loss 221108.51081666892
dev set
[0.40142176 0.39838848 0.40034195 0.4002526  0.40131023 0.40016715
 0.40193833 0.40006574 0.40121701 0.4004009 ]
[[1.11729834 0.81396821 1.00931317 1.00037786 1.10613139 1.04158479
  1.16895963 1.03145435 1.14658115 1.06496952]]
{0: 2717, 1: 97}
acc 0.9225302061122956
(0.35051546391752575, 0.17989417989417988, 0.23776223776223773, None)

test set
{0: 2616, 1: 86}
acc 0.9111769059955589
(0.37209302325581395, 0.14678899082568808, 0.2105263157894737, None)

batch-size: 2048 alpha-init: 0.5
precision penalty
0 loss 221216.8730794584
dev set
[0.50122477 0.49819146 0.50014488 0.50005552 0.50111312 0.50036412
 0.50174139 0.50026281 0.50141409 0.50059797]
[[1.11749531 0.81416524 1.0095103  1.00057494 1.10632855 1.04138782
  1.16915662 1.03125722 1.14638411 1.06477249]]
{0: 2701, 1: 113}
acc 0.9189765458422174
(0.3274336283185841, 0.19576719576719576, 0.24503311258278143, None)

test set
{0: 2603, 1: 99}
acc 0.9078460399703923
(0.3434343434343434, 0.15596330275

1 loss 221652.37932760434
dev set
[0.80127384 0.79824018 0.80019294 0.80010301 0.80116171 0.80031476
 0.80179067 0.80022536 0.80139509 0.80074444]
[[1.11744639 0.81411769 1.00948744 1.00055085 1.10628265 1.04143717
  1.16910745 1.03130351 1.14642695 1.06463055]]
{0: 2473, 1: 341}
acc 0.892679459843639
(0.3343108504398827, 0.6031746031746031, 0.430188679245283, None)

test set
{0: 2289, 1: 413}
acc 0.8782383419689119
(0.36561743341404357, 0.6926605504587156, 0.47860538827258314, None)

2 loss 221652.18341036356
dev set
[0.801323   0.79828923 0.8002412  0.80015092 0.80121077 0.80026552
 0.80183992 0.8001829  0.80136145 0.800793  ]
[[1.1173973  0.81406918 1.00945293 1.00051674 1.10623457 1.0414864
  1.16905824 1.03135065 1.14647246 1.06458392]]
{0: 2473, 1: 341}
acc 0.892679459843639
(0.3343108504398827, 0.6031746031746031, 0.430188679245283, None)

test set
{0: 2289, 1: 413}
acc 0.8782383419689119
(0.36561743341404357, 0.6926605504587156, 0.47860538827258314, None)

3 loss 221651.9874893

In [335]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4)

batch-size: 32
{1: 189, -1: 2625}
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sf

zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 10), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(10,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft Tensor("map/while/sft:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), d

zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 10), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(10,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f Tensor("map/while/Sum_1:0", shape=(), dtype=float64)
sft Tensor("map/while/sft:0", shape=(), dtype=float64)
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), d

In [338]:
for b in [32,64,128,512,1024,2048,4096]:
    print("batch-size:",b)
    train(0.0001,5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4)

batch-size: 32
{1: 189, -1: 2625}
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sf

t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 10), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(10,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f T

t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 10), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(10,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f T

t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision constraints penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj Tensor("map/while/Gather_2:0", shape=(?, 10), dtype=float64)
out Tensor("map/while/prec_zy/while/Select:0", shape=(10,), dtype=float64)
prec_z Tensor("map/while/Sum:0", shape=(), dtype=float64)
prec_t_pout Tensor("map/while/truediv:0", shape=(?, 1), dtype=float64)
f T

In [273]:
j = 2
k = tf.constant([[1,2,3,4]])
thetas = tf.constant([[1,2,3,4]])
l =  tf.constant([[1, 2, 3, 4],[3, 3, 3, 4]])
L_a = tf.constant([1,2])
print(l)
print("L_a",L_a)
print(l.shape[0])
# print(c)
# print(s)
# gc = tf.squeeze(gather_cols(s,[3]))
# print(gc)
# wh = tf.where(tf.equal(gc,2),tf.constant(True),tf.constant(False))
# w,g = tf.Session().run([wh,gc])
# print(w,g)
# m = tf.fill(
#     [1,7],
#     True,
#     name=None
# )
# tf.Session().run(m)
BATCH_SIZE = 2
Lj = gather_cols(l,[j])
kj = gather_cols(k,[j])
print(Lj)
print(kj)
mask = tf.squeeze(tf.equal(Lj,kj),[1])

print(mask)
print(l)
L_ij_equals_kj = tf.boolean_mask(l,mask)
print(L_ij_equals_kj)
prec_t_pout = tf.matmul(L_ij_equals_kj*kj, thetas,transpose_b=True)
print(prec_t_pout)
f = tf.reduce_sum(tf.expand_dims(L_a,1) - prec_t_pout)
sft = tf.nn.softplus(f,name="sft")
tf.Session().run([prec_t_pout,f,sft])
np.arange(5)

Tensor("Const_2:0", shape=(2, 4), dtype=int32)
L_a Tensor("Const_3:0", shape=(2,), dtype=int32)
2
<dtype: 'int32'>
<dtype: 'int32'>
<dtype: 'int32'>
<dtype: 'int32'>
Tensor("gather_cols/Reshape_3:0", shape=(?, ?), dtype=int32)
Tensor("gather_cols_1/Reshape_3:0", shape=(?, ?), dtype=int32)
Tensor("Squeeze:0", shape=(?,), dtype=bool)
Tensor("Const_2:0", shape=(2, 4), dtype=int32)
Tensor("boolean_mask/Gather:0", shape=(?, 4), dtype=int32)
Tensor("MatMul:0", shape=(?, 1), dtype=int32)


array([0, 1, 2, 3, 4])

In [78]:
l =  tf.constant([[1, 2, 3, 4],[3,4,5,6]])
mask = tf.constant([True,False])
print(l)
print(mask)
L_ij_equals_kj = tf.boolean_mask(l,mask)
print(L_ij_equals_kj)
tf.Session().run(L_ij_equals_kj)

Tensor("Const_107:0", shape=(2, 4), dtype=int32)
Tensor("Const_108:0", shape=(2,), dtype=bool)
Tensor("boolean_mask_9/Gather:0", shape=(?, 4), dtype=int32)


array([[1, 2, 3, 4]], dtype=int32)

In [224]:
j = 1

l =  tf.constant([[1, 2, 3, 4],[1, 3, 3, 4]])
k = tf.constant([1,2,3,4])
kj = tf.gather(k,j)
col = tf.map_fn(lambda li : tf.gather(li,j),l)
eq = tf.where(tf.equal(col,kj))
leqk = tf.gather(l,tf.squeeze(eq,1))
tf.Session().run([leqk,kj])

[array([[1, 2, 3, 4]], dtype=int32), 2]

In [188]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.name_scope(name, "gather_cols",[params, indices]) as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
            print(params.dtype)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
            print(indices.dtype)
        except ValueError:
            raise ValueError('\'indices\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [24]:
# dev data
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    R=train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),norm=False,smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -28920.421611836453
[ 0.00114834 -0.00192877  0.00101388  0.00092453  0.00101542 -0.0

3 loss -24324.46604019108
[0.20106773 0.19785987 0.20374722 0.20365786 0.20085259 0.19736642
 0.20385861 0.19666018 0.19781146 0.19699534]
[[1.11765234 0.8144968  1.00600269 0.99704372 1.10658905 1.04438557
  1.16708036 1.0347293  1.14987918 1.06830114]]
{0: 2737, 1: 77}
acc 0.9260838663823738
(0.37662337662337664, 0.15343915343915343, 0.21804511278195485, None)

4 loss -24441.946989478885
[0.20104121 0.19779079 0.2046652  0.20457585 0.20080344 0.19660371
 0.20440678 0.19574212 0.19689341 0.19607728]
[[1.11767887 0.81456588 1.00510857 0.99614376 1.10663821 1.04514829
  1.16654204 1.03561505 1.15077058 1.06920105]]
{0: 2739, 1: 75}
acc 0.9267945984363894
(0.38666666666666666, 0.15343915343915343, 0.2196969696969697, None)

[0.20104121 0.19779079 0.2046652  0.20457585 0.20080344 0.19660371
 0.20440678 0.19574212 0.19689341 0.19607728]
[[1.11767887 0.81456588 1.00510857 0.99614376 1.10663821 1.04514829
  1.16654204 1.03561505 1.15077058 1.06920105]]
{0: 2739, 1: 75}
acc 0.9267945984363894

1 loss -18233.865538499387
[0.50111848 0.49799827 0.50176535 0.50173301 0.50092744 0.49891183
 0.50250389 0.49857901 0.49973255 0.49892957]
[[1.1176016  0.81435841 1.00806904 0.99913532 1.1065142  1.04284018
  1.16857396 1.03264821 1.14776227 1.06624262]]
{0: 2707, 1: 107}
acc 0.9196872778962332
(0.32710280373831774, 0.18518518518518517, 0.23648648648648646, None)

2 loss -18288.958569170187
[0.5010902  0.49792721 0.50260451 0.50260117 0.50086141 0.49815974
 0.50291333 0.49770095 0.49885449 0.49805151]
[[1.11762987 0.81442947 1.00732011 0.99838763 1.10658024 1.04359232
  1.16825364 1.03337367 1.14848153 1.06700864]]
{0: 2712, 1: 102}
acc 0.9207533759772566
(0.3333333333333333, 0.17989417989417988, 0.23367697594501716, None)

3 loss -18344.90822574766
[0.50106198 0.49785655 0.50344601 0.50347196 0.50079687 0.49740717
 0.50332508 0.49682019 0.49797374 0.49717075]
[[1.11765809 0.81450013 1.00656911 0.99763778 1.10664478 1.04434493
  1.16793036 1.03410242 1.14920458 1.06777795]]
{0: 2714, 

0 loss -15617.795835581157
[0.80114662 0.79806672 0.80022123 0.80011771 0.8009654  0.79980673
 0.80167435 0.80015961 0.8013301  0.80070702]
[[1.11757345 0.81428996 1.00963296 1.0007069  1.10647625 1.04194538
  1.16922697 1.03140962 1.146522   1.06480009]]
{0: 2474, 1: 340}
acc 0.8923240938166311
(0.3323529411764706, 0.5978835978835979, 0.42722117202268434, None)

1 loss -15618.934387292413
[0.80111784 0.7979916  0.80034773 0.80022991 0.80086708 0.79919892
 0.80165777 0.8000064  0.80119611 0.80076598]
[[1.11760223 0.81436508 1.0097051  1.00078839 1.10657457 1.0425534
  1.16924785 1.03161153 1.14670955 1.06488057]]
{0: 2474, 1: 340}
acc 0.8923240938166311
(0.3323529411764706, 0.5978835978835979, 0.42722117202268434, None)

2 loss -15620.086200054015
[0.80108907 0.79791653 0.80047546 0.80034316 0.8007689  0.79858972
 0.80164134 0.79985185 0.801061   0.80082423]
[[1.11763101 0.81444015 1.00977615 1.00086895 1.10667275 1.0431628
  1.16926864 1.03181417 1.14689767 1.06496137]]
{0: 2474, 1: 3

In [18]:
# test data
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    R=train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),norm=True,smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 156285.81383787203
[ 0.0021718  -0.00087335  0.00105642  0.00096699  0.0020438  -0.00040336
  0.002555

4 loss 153672.85800301103
[0.20613531 0.20296937 0.20484324 0.20475341 0.20587159 0.19653585
 0.20575062 0.19571555 0.19687807 0.19605396]
[[1.11263618 0.80963519 1.00488472 0.99592636 1.10181117 1.04508319
  1.16489494 1.0355791  1.15071416 1.06915899]]
{0: 2635, 1: 67}
acc 0.9122871946706144
(0.3582089552238806, 0.11009174311926606, 0.16842105263157894, None)

[0.20613531 0.20296937 0.20484324 0.20475341 0.20587159 0.19653585
 0.20575062 0.19571555 0.19687807 0.19605396]
[[1.11263618 0.80963519 1.00488472 0.99592636 1.10181117 1.04508319
  1.16489494 1.0355791  1.15071416 1.06915899]]
{0: 2635, 1: 67}
acc 0.9122871946706144
(0.3582089552238806, 0.11009174311926606, 0.16842105263157894, None)
alpha-mean 0.30000000000000004
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_po

2 loss 153794.9386158779
[0.50405499 0.50066194 0.50273334 0.5027063  0.50363504 0.49822189
 0.50370149 0.49773228 0.49889572 0.49808706]
[[1.1148057  0.81213779 1.00708879 0.99813224 1.10419601 1.04350208
  1.16661865 1.03325522 1.1483165  1.06685048]]
{0: 2610, 1: 92}
acc 0.9089563286454478
(0.34782608695652173, 0.14678899082568808, 0.20645161290322583, None)

3 loss 153729.12681287836
[0.50501376 0.50149986 0.50361548 0.50360996 0.50449111 0.49749125
 0.50436683 0.49686138 0.49802773 0.497217  ]
[[1.11389455 0.81144762 1.00626364 0.99730046 1.10346894 1.04422403
  1.16575989 1.03394436 1.14898469 1.06756728]]
{0: 2614, 1: 88}
acc 0.9096965210954848
(0.3522727272727273, 0.14220183486238533, 0.20261437908496732, None)

4 loss 153662.77625269737
[0.50597217 0.50233706 0.50449881 0.5045151  0.50534654 0.49676078
 0.50502984 0.4959878  0.49715699 0.49634424]
[[1.11298385 0.81075771 1.00543795 0.99646821 1.10274206 1.04494612
  1.16490309 1.0346367  1.1496567  1.06828743]]
{0: 2619, 1: 83

marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 154383.66560828217
[0.80178968 0.79840525 0.80032777 0.80022986 0.80131478 0.79985738
 0.80241745 0.80023172 0.80141595 0.8008523 ]
[[1.11707354 0.81405804 1.00951065 1.0005744  1.10624022 1.04183987
  1.16838791 1.03134809 1.14644999 1.06457379]]
{0: 2294, 1: 408}
acc 0.8808290155440415
(0.37254901960784315, 0.6972477064220184, 0.4856230031948882, None)

1 loss 154382.0941822927
[0.80238877 0.79865261 0.8005598  0.8004528  0.80156025 0.79930028
 0.8031278  0.80015051 0.80136747 0.80105686]
[[1.11662259 0.81391978 1.00946195 1.00052513 1.10610856 1.0423427
  1.16757512 1.03148887 1.14656611 1.064428  ]]
{0: 2294, 1: 408}
acc 0.8808290155440415
(0.37254901960784315, 0.6972477064220184, 0.4856230031948882, None)

2 loss 154380.52067854622
[0.80298586 0.79889913 0.80079239 0.80067612 0.80180505 0.79874205
 0.80383562 0.80006818 0.80131807 0.80126116]
[[1.11617333 0.81378194 1.0094127  1.00047544 1.10597718 1.0428471


In [19]:
#test data
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    R=train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),norm=False,smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -28920.421611836453
[ 0.00114834 -0.00192877  0.00101388  0.00092453  0.00101542 -0.0

3 loss -24324.46604019108
[0.20106773 0.19785987 0.20374722 0.20365786 0.20085259 0.19736642
 0.20385861 0.19666018 0.19781146 0.19699534]
[[1.11765234 0.8144968  1.00600269 0.99704372 1.10658905 1.04438557
  1.16708036 1.0347293  1.14987918 1.06830114]]
{0: 2633, 1: 69}
acc 0.9115470022205774
(0.34782608695652173, 0.11009174311926606, 0.16724738675958187, None)

4 loss -24441.946989478885
[0.20104121 0.19779079 0.2046652  0.20457585 0.20080344 0.19660371
 0.20440678 0.19574212 0.19689341 0.19607728]
[[1.11767887 0.81456588 1.00510857 0.99614376 1.10663821 1.04514829
  1.16654204 1.03561505 1.15077058 1.06920105]]
{0: 2635, 1: 67}
acc 0.9122871946706144
(0.3582089552238806, 0.11009174311926606, 0.16842105263157894, None)

[0.20104121 0.19779079 0.2046652  0.20457585 0.20080344 0.19660371
 0.20440678 0.19574212 0.19689341 0.19607728]
[[1.11767887 0.81456588 1.00510857 0.99614376 1.10663821 1.04514829
  1.16654204 1.03561505 1.15077058 1.06920105]]
{0: 2635, 1: 67}
acc 0.9122871946706144

1 loss -18233.865538499387
[0.50111848 0.49799827 0.50176535 0.50173301 0.50092744 0.49891183
 0.50250389 0.49857901 0.49973255 0.49892957]
[[1.1176016  0.81435841 1.00806904 0.99913532 1.1065142  1.04284018
  1.16857396 1.03264821 1.14776227 1.06624262]]
{0: 2606, 1: 96}
acc 0.9074759437453738
(0.3333333333333333, 0.14678899082568808, 0.20382165605095542, None)

2 loss -18288.958569170187
[0.5010902  0.49792721 0.50260451 0.50260117 0.50086141 0.49815974
 0.50291333 0.49770095 0.49885449 0.49805151]
[[1.11762987 0.81442947 1.00732011 0.99838763 1.10658024 1.04359232
  1.16825364 1.03337367 1.14848153 1.06700864]]
{0: 2610, 1: 92}
acc 0.9089563286454478
(0.34782608695652173, 0.14678899082568808, 0.20645161290322583, None)

3 loss -18344.90822574766
[0.50106198 0.49785655 0.50344601 0.50347196 0.50079687 0.49740717
 0.50332508 0.49682019 0.49797374 0.49717075]
[[1.11765809 0.81450013 1.00656911 0.99763778 1.10664478 1.04434493
  1.16793036 1.03410242 1.14920458 1.06777795]]
{0: 2613, 1:

0 loss -15617.795835581157
[0.80114662 0.79806672 0.80022123 0.80011771 0.8009654  0.79980673
 0.80167435 0.80015961 0.8013301  0.80070702]
[[1.11757345 0.81428996 1.00963296 1.0007069  1.10647625 1.04194538
  1.16922697 1.03140962 1.146522   1.06480009]]
{0: 2294, 1: 408}
acc 0.8808290155440415
(0.37254901960784315, 0.6972477064220184, 0.4856230031948882, None)

1 loss -15618.934387292413
[0.80111784 0.7979916  0.80034773 0.80022991 0.80086708 0.79919892
 0.80165777 0.8000064  0.80119611 0.80076598]
[[1.11760223 0.81436508 1.0097051  1.00078839 1.10657457 1.0425534
  1.16924785 1.03161153 1.14670955 1.06488057]]
{0: 2294, 1: 408}
acc 0.8808290155440415
(0.37254901960784315, 0.6972477064220184, 0.4856230031948882, None)

2 loss -15620.086200054015
[0.80108907 0.79791653 0.80047546 0.80034316 0.8007689  0.79858972
 0.80164134 0.79985185 0.801061   0.80082423]
[[1.11763101 0.81444015 1.00977615 1.00086895 1.10667275 1.0431628
  1.16926864 1.03181417 1.14689767 1.06496137]]
{0: 2294, 1: 4

In [141]:
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.7,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0,alp=0.8)

Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 154419.85892351693
[0.70199849 0.69863948 0.70064995 0.70050634 0.70158368 0.69974811
 0.7024189  0.69989863 0.70116149 0.70031883]
[[1.11684702 0.81388424 1.00909

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [160]:
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.7,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),norm=False,\
                          smooth=True,penalty=2,alp=0.8)

Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -15939.431711918833
[0.70114658 0.6980671  0.7005352  0.70036628 0.70097048 0.69970941
 0.70172842 0.6998

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = np.load("train_L_S_discrete.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)

L_gold = np.load("test_L_S_discrete.npy")
L_gold = L_gold[:,0,:].astype(int)
print(np.array(L_gold).shape)
L_gold = sp.csr_matrix(L_gold)

from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))